In [1]:
"""
Grover's Search Benchmark Program - Qiskit
"""

import sys
import time
from qiskit import *
from qiskit import IBMQ, Aer
from qiskit.visualization import plot_histogram
from qiskit.providers.aer.noise import NoiseModel

import numpy as np
from pytket import *
import pytket.extensions.qiskit as pyqi

import pytket.extensions.qsharp as AB



In [2]:
# provider = IBMQ.load_account()
# backend = provider.get_backend('ibmq_lima')
"""Basic Qiskit circuit with no TKET adjustment"""

qc = QuantumCircuit(2,2)
qc.h(0)
qc.cx(0,1)

meas = QuantumCircuit(2,2)
meas.measure([0,1],[0,1])

regularBackend = BasicAer.get_backend('qasm_simulator')
drawing = qc.compose(meas)
result = regularBackend.run(transpile(drawing, regularBackend), shots=1000).result()
counts = result.get_counts(qc)
print(counts)



{'11': 510, '00': 490}


In [3]:
"""Qiskit to TKET conversion """

tketCirc = pyqi.qiskit_to_tk(qc)
tketCirc.measure_all()
print(type(tketCirc))
tketQiskitBackend = pyqi.AerBackend()
tket_compiled_circ = tketQiskitBackend.get_compiled_circuit(tketCirc)
tket_handle = tketQiskitBackend.process_circuit(tketCirc,n_shots=5)
tket_counts = tketQiskitBackend.get_result((tket_handle)).get_counts()
print(tket_counts)


<class 'pytket._tket.circuit.Circuit'>
Counter({(0, 0): 4, (1, 1): 1})


In [4]:
IONQ = AB.AzureBackend("ionq.simulator", resourceId = "/subscriptions/7ef9c9ba-198a-420f-a70b-c4341dd14797/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/IONQTest1forChoi", location="East US", storage="aqa259c256c06c4eac875c34")
print(IONQ.valid_circuit(tketCirc))
print("Pass 1")
compiled_cric = IONQ.get_compiled_circuit(tketCirc)
print("Pass 2")
processCircuit = IONQ.process_circuit(compiled_cric,n_shots=5)
print("Pass 3")
tketResult = IONQ.get_result(processCircuit)

print(tketResult)

Connected to Azure Quantum workspace IONQTest1forChoi in location eastus.
Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator
True
Pass 1
Pass 2
Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


Exception has been thrown by the target of an invocation.


TypeError: argument of type 'NoneType' is not iterable